In [7]:
import networkx as nx
import numpy as np
import pandas as pd
import pymatgen
import pymatgen.io.xyz
import itertools as it
#targets = ((6,7,7,6,5,5), (7,7,5,5,6,6), (7,7,5,6,5,6), (7,6,7,5,5,6), (5,6,5,7,6,7), (6,6,7,5,5,7), (6,6,5,7,7,5), (7,6,5,7,5,6), (5, 6, 7, 5, 7, 6), (6,6,7,5,7,5), (6,7,5,6,7,5))

In [2]:
def detect_automorphisms(filename):
    xyz = pymatgen.io.xyz.XYZ.from_file(filename)
    psa = pymatgen.symmetry.analyzer.PointGroupAnalyzer(xyz.molecule)
    
    m = xyz.molecule.get_centered_molecule()
    carbons = np.where(np.array(m.atomic_numbers, dtype=np.int) == 6)[0]
    
    operations = psa.get_symmetry_operations()
    mapping = np.zeros((len(carbons), len(operations)), dtype=np.int)
    for opidx, op in enumerate(operations):
        for bidx, base in enumerate(carbons):
            ds = np.linalg.norm(op.operate(m.cart_coords[base]) - m.cart_coords[carbons], axis=1)
            onto = np.argmin(ds)
            if ds[onto] > 1e-3:
                raise ValueError('Irregular geometry')
            mapping[bidx, opidx] = onto
    
    return mapping
def two_at_once(based_on, mappings):
    modifiable = sorted(np.where(np.array(based_on) == 6)[0])
    for i in modifiable:
        for j in modifiable:
            if i == j:
                continue
            zsnew = np.array(based_on).copy()
            zsnew[i] = 5
            zsnew[j] = 7
            
            if is_canonical(zsnew, mappings):
                yield zsnew
def make_canonical(zs, mappings):
    zs = np.array(zs)
    #if len(np.where(zs == 7)[0]) > len(np.where(zs == 5)[0]):
    #    return False
    permutations = zs[mappings].T
    o = np.lexsort(permutations.T)
    canonical = permutations[o[0]]
    return canonical
def is_canonical(zs, mappings):
    return (zs == make_canonical(zs, mappings)).all()

In [3]:
def enumerate_next_bn(based_on):
    mappings = detect_automorphisms("naphtalene.xyz")
    
    res = []
    for k in two_at_once(based_on, mappings):
        res.append(tuple(k))
    return set(res)

In [4]:
bn1 = enumerate_next_bn((6,6,6,6,6,6,6,6,6,6))
bn2 = set.union(*[enumerate_next_bn(_) for _ in bn1])
bn3 = set.union(*[enumerate_next_bn(_) for _ in bn2])
bn4 = set.union(*[enumerate_next_bn(_) for _ in bn3])
bn5 = set.union(*[enumerate_next_bn(_) for _ in bn4])

In [5]:
len(bn1), len(bn2), len(bn3), len(bn4), len(bn5)

(23, 320, 1026, 801, 66)

In [9]:
found = []
mappings = detect_automorphisms("naphtalene.xyz")
for combo in it.permutations((5,7,5,7,6,6,6,6,6,6)):
    if is_canonical(combo, mappings):
        found.append(tuple(combo))

In [21]:
# ref: 23, 330, 1056, 810, 66

2236

In [15]:
set(found) - bn2

{(5, 5, 6, 6, 6, 6, 6, 7, 7, 6),
 (5, 5, 6, 7, 7, 6, 6, 6, 6, 6),
 (5, 6, 6, 6, 5, 6, 6, 7, 7, 6),
 (5, 6, 6, 6, 6, 5, 6, 7, 7, 6),
 (6, 5, 6, 6, 5, 6, 6, 7, 7, 6),
 (6, 5, 6, 6, 6, 5, 6, 7, 7, 6),
 (6, 6, 5, 6, 6, 5, 6, 7, 7, 6),
 (6, 6, 6, 5, 5, 6, 6, 7, 7, 6),
 (6, 6, 6, 5, 6, 5, 6, 7, 7, 6),
 (6, 6, 6, 6, 5, 5, 6, 7, 7, 6)}

In [16]:
bn1

{(5, 6, 6, 6, 6, 6, 7, 6, 6, 6),
 (5, 6, 7, 6, 6, 6, 6, 6, 6, 6),
 (6, 5, 6, 6, 6, 6, 7, 6, 6, 6),
 (6, 5, 7, 6, 6, 6, 6, 6, 6, 6),
 (6, 6, 5, 6, 6, 6, 7, 6, 6, 6),
 (6, 6, 6, 5, 6, 6, 7, 6, 6, 6),
 (6, 6, 6, 6, 5, 6, 7, 6, 6, 6),
 (6, 6, 6, 6, 6, 5, 7, 6, 6, 6),
 (6, 6, 6, 6, 6, 6, 6, 6, 7, 5),
 (6, 6, 6, 6, 6, 6, 6, 7, 6, 5),
 (6, 6, 6, 6, 6, 6, 7, 6, 6, 5),
 (6, 6, 6, 6, 6, 7, 6, 6, 6, 5),
 (6, 6, 6, 6, 7, 5, 6, 6, 6, 6),
 (6, 6, 6, 6, 7, 6, 6, 6, 6, 5),
 (6, 6, 6, 7, 6, 5, 6, 6, 6, 6),
 (6, 6, 6, 7, 6, 6, 6, 6, 6, 5),
 (6, 6, 7, 6, 6, 5, 6, 6, 6, 6),
 (6, 6, 7, 6, 6, 6, 6, 6, 6, 5),
 (6, 7, 6, 6, 6, 5, 6, 6, 6, 6),
 (6, 7, 6, 6, 6, 6, 6, 6, 6, 5),
 (7, 5, 6, 6, 6, 6, 6, 6, 6, 6),
 (7, 6, 6, 6, 6, 5, 6, 6, 6, 6),
 (7, 6, 6, 6, 6, 6, 6, 6, 6, 5)}

In [17]:
12005168*90/60/60

300129.2

In [29]:
graph = nx.Graph()
for i in range(10):
    graph.add_node(i)
for i in range(5):
    print (i, i+1)
    graph.add_edge(i, i+1)
graph.add_edge(5, 0)
for i in range(5, 9):
    print (i, i+1)
    graph.add_edge(i, i+1)
graph.add_edge(4, 9)

0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9


In [23]:
nx.generate_graph6(graph, header=False)

AttributeError: module 'networkx' has no attribute 'generate_graph6'

In [30]:
nx.write_graph6(graph, "naphtalene.g6")

In [61]:
#/vcolg -m3 -T /mnt/c/Users/guido/workcopies/apdft/prototyping/networks/benzene.g6 -q > list
with open("/mnt/c/Users/guido/opt/nauty/nauty27rc3/list") as fh:
    count = 0
    for line in fh:
        parts = line.strip().split()
        parts = parts[2:2+int(parts[1])]
        bs = len([_ for _ in parts if _ == "1"])
        ns = len([_ for _ in parts if _ == "2"])
        if ns == bs:
            count += 1
count

2286

In [32]:
import MDAnalysis as mda

In [58]:
def mol2_to_graph6(number):
    parsed = mda.topology.MOL2Parser.MOL2Parser("db/%d.mol2" % number).parse()
    carbons = np.where(parsed.names.values == 'C')[0]
    print (carbons)
    
    graph = nx.Graph()
    for carbon in carbons:
        graph.add_node(carbon)
        
    bonds = parsed.bonds.values
    for bond in bonds:
        if bond[0] in carbons and bond[1] in carbons:
            graph.add_edge(*bond)
            print (bond)
    
    nx.write_graph6(graph, "db/%d.g6" % number)
mol2_to_graph6(1)

[0 1 2 3 4 5 6 7 8 9]
(0, 1)
(0, 2)
(0, 4)
(1, 3)
(1, 5)
(2, 6)
(3, 7)
(4, 8)
(5, 9)
(6, 7)
(8, 9)


array(['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C',
       'C', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'],
      dtype=object)